In [0]:
creator_catalog = 'komae_cleanroom'
collaborator_catalog = 'komae_cleanroom'

### ID突合

`customer`と`sales`の顧客データをハッシュ化された顧客IDで結合します。  
あなたとコラボレーターが同じハッシュアルゴリズムを使用してメールアドレスを正規化していることを前提とします。

**注意事項**

- あなたは `customer` テーブルをアップロードし、コラボレーターは `sales` テーブルをアップロードします。

In [0]:
spark.sql(
  f"""
    CREATE OR REPLACE TEMP VIEW v_matched_customers AS (
        SELECT
          od.*,
          cu.city,
          cu.state
        FROM {creator_catalog}.creator.customers cu
        INNER JOIN {collaborator_catalog}.collaborator.orders od
        -- INNER JOIN {collaborator_catalog}.creator.orders od
        ON cu.customerid_sha256 = od.customerid_sha256
    )
  """
)

In [0]:
%sql

select * from v_matched_customers limit 10

### 結果を出力カタログに保存

In [0]:
CREATE TABLE identifier(
  :cr_output_catalog ||
  '.' ||
  :cr_output_schema ||
  '.overlapping_customers_sales'
) AS
SELECT *
FROM v_matched_customers

In [0]:
# if output_catalog and output_schema:
#     spark.sql(
#         f"""
#         -- CREATE TABLE {output_catalog}.{output_schema}.overlapping_households AS
#         CREATE TABLE overlapping_households AS
#         SELECT DISTINCT household_id from adv_pub_overlap
#         """
#     )
# else:
#     print("出力カタログまたはスキーマが指定されていません。テーブル作成をスキップします。")